In [43]:
# Tip: By now, install transformers from source
from transformers import AutoModelWithLMHead, AutoTokenizer
import pandas as pd 

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/opt/anaconda3/envs/NLP/lib/python3.7/site-packages/transformers/models/auto/modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [44]:
def get_question(answer, context, max_length=64):
    input_text = "answer: %s  context: %s </s>" % (answer, context)
    features = tokenizer([input_text], return_tensors='pt')

    output = model.generate(input_ids=features['input_ids'], 
                  attention_mask=features['attention_mask'],
                  max_length=max_length)

    return tokenizer.decode(output[0])

In [45]:
def qa_generation(context):
    print("context: " + context)
    answer = input("answer: ")
    print(answer)
    question = get_question(answer, context)[16:-4]
    print("question: " + question)
    add_question = input("add some word in questions : ")
    difficulty = input("difficulty(e,m,h,f): ")
    print(difficulty + '\n')

    return answer, question + '' + add_question, difficulty.upper()

In [46]:
def sentence_extract(dir):
    document = [] 
    with open("documents"+dir, "r") as file:
        while True:
            line = file.readline()
            if not line:
                break
            if line != '\n' and line[0] != '=':
                sentence = line.strip('\n').split(". ")
                document.extend(sentence)
    
    return document

def sentence_extract2(dir):
    document = [] 
    with open("documents"+dir, "r") as file:
        while True:
            line = file.readline()
            if not line:
                break
            if line != '\n' and line[0] != '=':
                sentence = line.strip('\n').split(". ")
                document.extend(sentence)

    two_sentence = ''
    new_document = []
    for i in document:
        if two_sentence == '':
            two_sentence = i
        else:
            new_document.append(two_sentence + '. ' + i + '.')
            two_sentence = ''
    
    return new_document

def sentence_extract3(dir):
    document = [] 
    with open("documents"+dir, "r") as file:
        while True:
            line = file.readline()
            if not line:
                break
            if line != '\n' and line[0] != '=':
                sentence = line.strip('\n').split(". ")
                document.extend(sentence)

    num_sentence = 0
    sentence = ''
    new_document = []
    for i in document:
        if num_sentence == 0:
            sentence = i
            num_sentence += 1
        elif num_sentence < 2:
            sentence = sentence + '. ' + i
            num_sentence += 1
        else:
            new_document.append(sentence + '. ' + i)
            sentence = ''
            num_sentence = 0
    
    return new_document
dir = "/languages/Mandarin_language.txt"
document = sentence_extract3(dir)

In [47]:
len(document)

89

In [50]:
answers = []
questions = []
documents = []
difficulties = []
for i, sentence in enumerate(document):
    if i < 20:
        continue
    elif i == 22:
        break
    a, q, d = qa_generation(sentence)

    if d =='F':
        continue
        
    answers.append(a)
    questions.append(q)
    difficulties.append(d) # E, M, H

context: Knowledge of this language was thus essential for an official career, but it was never formally defined.Officials varied widely in their pronunciation; in 1728, the Yongzheng Emperor, unable to understand the accents of officials from Guangdong and Fujian, issued a decree requiring the governors of those provinces to provide for the teaching of proper pronunciation. Although the resulting Academies for Correct Pronunciation (正音書院; Zhèngyīn Shūyuàn) were short-lived, the decree did spawn a number of textbooks that give some insight into the ideal pronunciation. Common features included:
no
question: Was pronunciation formally defined?
h

context: loss of the Middle Chinese voiced initials except for v-. merger of -m finals with -n. the characteristic Mandarin four-tone system in open syllables, but retaining a final glottal stop in "entering tone" syllables
no
question: Does the Mandarin four-tone system have a final glottal stop?
f



In [51]:
dict = {'document_id': dir, 'question': questions, 'answer': answers, 'difficulty': difficulties}  
df = pd.DataFrame(dict) 
df.to_csv('test.csv', index=False)  
#df.to_csv('test.tsv', index=False, sep="\t") 

In [52]:
df1 = pd.read_csv ('result_4.csv')

In [53]:
df1['difficulty'].value_counts()

M    40
E    30
H    30
Name: difficulty, dtype: int64